This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [1]:
structure_name = "LLZO_001_Zr_code93_sto__Li_111_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_001_Zr_code93_sto__Li_111_slab_heavy
- Initial lattice height: 74.67 Å
- Number of atoms: 1044


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 20.26 Å
Lowest Li slab atom: 39.40 Å
LLZO top z: 35.40 Å
Li penetration into LLZO: -4.00 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 20.40 Å
Li slab thickness:   20.26 Å
LLZO top z:          35.40 Å
Lowest Li atom z:    39.40 Å
Li penetration into LLZO: -4.00 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 794 atoms out of 1044
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_158292/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 13:26:40    -2573.896995        3.907484
SciPyFminCG:    1 13:26:44    -2590.718265        2.847568
SciPyFminCG:    2 13:26:49    -2613.195743        4.846782
SciPyFminCG:    3 13:26:53    -2629.035316        5.247977
SciPyFminCG:    4 13:26:57    -2634.100863       12.449854
SciPyFminCG:    5 13:27:00    -2635.199547       10.334740
SciPyFminCG:    6 13:27:04    -2637.731449        4.662235
SciPyFminCG:    7 13:27:11    -2641.486988        5.314781
SciPyFminCG:    8 13:27:15    -2644.745207        2.657981
SciPyFminCG:    9 13:27:18    -2647.174061        5.886053
SciPyFminCG:   10 13:27:20    -2647.776172        3.052321
SciPyFminCG:   11 13:27:24    -2648.505724        1.747678
SciPyFminCG:   12 13:27:28    -2648.913935        1.687279
SciPyFminCG:   13 13:27:31    -2649.962339        2.363280
SciPyFminCG:   14 13:27:37    -2650.951753        1.183486
SciPyFminCG:   15 13:27:40    -2651.428413        2.138244

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_158292/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 13:39:37    -2704.920699        0.147028
FIRE:    1 13:39:39    -2704.924184        0.116145
FIRE:    2 13:39:40    -2704.928415        0.086095
FIRE:    3 13:39:42    -2704.929909        0.128087
FIRE:    4 13:39:44    -2704.931900        0.125998
FIRE:    5 13:39:46    -2704.936131        0.102992
FIRE:    6 13:39:48    -2704.940114        0.086363
FIRE:    7 13:39:50    -2704.944843        0.083573
FIRE:    8 13:39:51    -2704.949821        0.132418
FIRE:    9 13:39:53    -2704.955795        0.098697
FIRE:   10 13:39:55    -2704.962516        0.100816
FIRE:   11 13:39:57    -2704.969236        0.092263
FIRE:   12 13:39:58    -2704.978944        0.121644
FIRE:   13 13:40:00    -2704.990394        0.147275
FIRE:   14 13:40:02    -2705.004581        0.244054
FIRE:   15 13:40:04    -2705.023250        0.274855
FIRE:   16 13:40:06    -2705.049883        0.441872
FIRE:   17 13:40:07    -2705.049385        0.892451
FIRE:   18 13:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")
# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 24.60 Å
Lowest Li slab atom: 35.07 Å
LLZO top z: 34.90 Å
Li penetration into LLZO: -0.16 Å


In [10]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)


             Step     Time          Energy          fmax
SciPyFminCG:    0 13:42:33    -2705.449382        3.865400
SciPyFminCG:    1 13:42:38    -2730.148124        5.031102
SciPyFminCG:    2 13:42:42    -2759.325485        5.310499
SciPyFminCG:    3 13:42:45    -2788.753000        5.810030
SciPyFminCG:    4 13:42:49    -2814.216391        3.796720
SciPyFminCG:    5 13:42:52    -2837.162813        3.813039
SciPyFminCG:    6 13:42:55    -2855.301807        3.149445
SciPyFminCG:    7 13:42:59    -2865.304962        3.737618
SciPyFminCG:    8 13:43:02    -2874.230839        4.510073
SciPyFminCG:    9 13:43:05    -2884.193172        3.927371
SciPyFminCG:   10 13:43:08    -2895.982000        1.848064
SciPyFminCG:   11 13:43:12    -2903.225501        1.675083
SciPyFminCG:   12 13:43:15    -2907.693169        2.077600
SciPyFminCG:   13 13:43:18    -2913.053175        1.497651
SciPyFminCG:   14 13:43:21    -2916.867456        1.798843
SciPyFminCG:   15 13:43:25    -2920.655105        1.562932

In [11]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_158292/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 14:02:24    -3129.166918        0.512466
FIRE:    1 14:02:26    -3129.185835        0.367083


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.419508199475576e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:    2 14:02:27    -3129.207241        0.192829


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.4448788625674685e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


...

FIRE:  398 14:11:09    -3133.664703        0.125316


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.4624104874745756e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  399 14:11:10    -3133.675158        0.305838


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.423169804215582e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  400 14:11:11    -3133.682127        0.463514


In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_001_Zr_code93_sto__Li_111_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 